In [ ]:
%pip install -U --quiet langchain-google-genai

In [30]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from google.generativeai.types.safety_types import HarmBlockThreshold, HarmCategory
import os

- pdf용

In [12]:
## 1. 로더(PDF시 PDF리더, TEXT시 단순 텍스트)
### 로더는 종류별로 각종 메타데이터(텍스트 이외의 정보 ex)페이지 번호 등)
loader = PyPDFLoader("한국어 4-24 토끼전.pdf")
pages = loader.load_and_split() ## 페이지별로 찢기

- 동화 종류

In [4]:
tale_dir = os.getcwd() + "/동화책"
tale_text_list = []

print(os.listdir(tale_dir))

for i in os.listdir(tale_dir) :    
    with open(tale_dir + "/" + i, 'r') as file:
        # # 파일의 모든 내용을 읽어서 문자열로 반환합니다.
        file_content = file.read()
        tale_text_list.append(file_content)

print("완료")

['견우와 직녀.txt', '금도끼 은도끼.txt', '단군 이야기.txt', '바보온달.txt', '콩쥐팥쥐.txt', '토끼와 거북이.txt', '토끼전.txt', '해와 달이 된 오누이.txt', '호랑이와 곶감.txt', '혹부리 영감.txt']
완료


In [5]:
## 텍스트 길이 => chunk의 필요는 없다
for i in tale_text_list:
    print(len(i))

657
669
701
715
910
604
722
585
593
852


In [8]:
# ## 2. 토크나이저(필요가 없다)
# text_splitter = RecursiveCharacterTextSplitter(separator = "\n\n", chunk_size=500, chunk_overlap=50, length_function = len) 
# texts = text_splitter.split_documents(pages)

In [6]:
# 3. 임베딩기 선정(Open ai 또는 Hugging face)
model_name = "jhgan/ko-sbert-nli" # 3. 임베딩기
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [8]:
## 4. VectorDB(Chroma)
DB_PATH = os.getcwd() + "/DB"
# docsearch = Chroma.from_texts(tale_text_list, hf, persist_directory=DB_PATH) 

## 4-1 DB통해 부르기
docsearch = Chroma(persist_directory=DB_PATH, embedding_function=hf)

In [9]:
## 5. 검색기
retriever = docsearch.as_retriever(
                                search_type="mmr",
                                search_kwargs={'k':1, 'fetch_k': 100}) ## k는 검색 유사도 문장 수
retriever.get_relevant_documents("제목 : 바보온달") ## 이 후 chain.invoke question이 들어올 곳

Number of requested results 100 is greater than number of elements in index 10, updating n_results = 10


[Document(page_content='제목 : 바보온달\n\n옛날 고구려에 예쁜 평강 공주가 살았어요. \n그런데 평강 공주는 어렸을 때부터 울보였어요.\n\n임금님은 평강 공주가 울 때마다 말했어요. \n“그렇게 울면 바보 온달과 결혼하게 될 거야.”\n\n평강 공주는 예쁘게 자라서 결혼할 나이가 되었어요. \n평강 공주는 임금님에게 말했어요.\n“저는 바보 온달과 결혼할 거예요.”\n\n임금님은 화가 났어요.\n그래서 평강 공주를 궁궐 밖으로 쫓아냈어요. \n평강 공주는 바보 온달을 찾아갔어요.\n\n평강 공주는 바보 온달에게\n글쓰기, 활쏘기, 말타기를 가르쳤어요. \n바보 온달은 열심히 배워서 똑똑하고 씩씩한 사람이 되었어요.\n\n그러던 어느 날이었어요. \n이웃 나라가 고구려를 공격하기 시작했어요.\n사람들이 모두 도망갔어요. \n\n온달은 사람들에게 말했어요. \n“도망가지 말고 우리 같이 싸웁시다.”\n\n평강 공주는 온달에게 칼과 갑옷을 주었어요.\n\n온달은 전쟁에 나가 싸웠어요. \n온달은 이웃 나라 장군을 이겼어요.\n그리고 이웃 나라 군사들은 온달이 무서워서 도망갔어요.\n\n사람들은 모두 온달을 칭찬했어요.\n\n임금님이 온달을 궁궐로 불러 물었어요.\n\n“네 이름이 무엇이냐?”\n “제 이름은 온달입니다.”\n임금님은 온달의 씩씩한 모습에 깜짝 놀랐어요. \n온달은 바보가 아니라 아주 씩씩한 사람이었어요. \n임금님은 온달에게 많은 상을 주었어요. \n온달은 고구려의 훌륭한 장군이 되었어요.\n그리고 평강 공주와 결혼해서 행복하게 살았어요.\n\n')]

In [65]:
## 프롬프트
template = """Make three quizzes of three choices for {years} years old korean kids, based on the fairy tale given below And never give another answer and return the quiz in json form
Let me show you an example. It'just example.

example:
"question" : "용왕님이 토끼에게 필요한 것은?",
"option" : ["간","금덩이","귀"],
"answer" : 0

option must be a short for kids

fairy tale:
{context}


Avoid the use of long sentences
Remember that You must put like [quiz, quiz, quiz]
"""

prompt = ChatPromptTemplate.from_template(template)

# harm_categories = {
#     HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_LOW,
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM,
#     HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_HIGH,
#     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM,
#     HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_HIGH
# }


## 6. LLM 선정
os.environ['GOOGLE_API_KEY'] = ""
gemini = ChatGoogleGenerativeAI(model="gemini-pro", temperature = 0.5)

## RAG
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents("제목 : " + x['topic']),
    "years" : lambda x:x['years']
}) | prompt | gemini

# 견우와 직녀/바보 온달 제외
print(type(chain.invoke({"topic": "금도끼 은도끼", "years":"4"}).content))

Number of requested results 100 is greater than number of elements in index 10, updating n_results = 10


<class 'str'>


[
  {
    "question": "혹부리 영감이 산에 갔던 이유는?",
    "option": ["나무를 하러", "노래를 부르러", "도깨비를 만나러"],
    "answer": 0
  },
  {
    "question": "도깨비들이 혹부리 영감에게 떼어간 것은?",
    "option": ["귀", "코", "혹"],
    "answer": 2
  },
  {
    "question": "마음씨 나쁜 혹부리 영감에게 도깨비들이 준 것은?",
    "option": ["보물", "혹", "노래주머니"],
    "answer": 1
  }